In [ ]:
import openrouteservice as ors
import pandas as pd
import numpy as np
import folium 
import json
import csv
import time
import requests
import googlemaps
import os

In [ ]:
# Load amenities stats data
amenities_stats = pd.read_csv("../../datasets/property/property_nearest_amenities.csv")

# Pivot amenities per property
amenities_pivot = amenities_stats.pivot_table(
    index=['Property_Lat', 'Property_Lon', 'Address'],
    columns='Amenity_Type',
    values=['Amenity_Lat', 'Amenity_Lon'],
    aggfunc='first'
).reset_index()

# Flatten MultiIndex columns
amenities_pivot.columns = ['_'.join(filter(None, col)).strip('_') for col in amenities_pivot.columns]
amenities_pivot.head()

### Try Open Route Service

In [ ]:
# CONFIG
ORS_KEY = ''

CBD_COORDS = (144.9671, -37.8183)  
BATCH_DELAY = 1         
MAX_RETRIES = 3
MAX_ROUTES = 3500 

INPUT_CSV = amenities_pivot
OUTPUT_CSV = "../../datasets/property/property_with_distances_copy.csv"


# initialize client

client = ors.Client(key=ORS_KEY)

# Utility func
def chunk_list(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

def safe_distance_matrix(origins, destinations):
    """Compute ORS distance matrix with retries"""
    for attempt in range(MAX_RETRIES):
        try:
            res = client.distance_matrix(
                locations=origins + destinations,
                profile='driving-car',
                sources=list(range(len(origins))),
                destinations=list(range(len(origins), len(origins) + len(destinations))),
                metrics=['distance', 'duration']
            )
            return res
        except Exception as e:
            print(f"ORS attempt {attempt+1} failed: {e}. Retrying in 5s...")
            time.sleep(5)
    print("ORS failed after retries. Returning NaN.")
    n = len(destinations)
    return {'distances': [[float('nan')]*n for _ in origins],
            'durations': [[float('nan')]*n for _ in origins]}


# load/create output
if os.path.exists(OUTPUT_CSV):
    distance_df = pd.read_csv(OUTPUT_CSV)
    
    # Only consider rows with valid CBD distance as completed
    completed_indices = set(distance_df[~distance_df['Distance_to_CBD_m'].isna()].index)
    print(f"Resuming from {len(completed_indices)} completed properties...")
else:
    distance_df = pd.DataFrame()
    completed_indices = set()

# Prepare origins and destinations
origins_list = []
property_indices = []
destinations_set = set()
for idx, row in amenities_pivot.iterrows():
    if idx in completed_indices:
        continue  
    origins_list.append((row['Property_Lon'], row['Property_Lat']))
    property_indices.append(idx)
    
    if pd.notna(row.get('Amenity_Lon_supermarket')) and pd.notna(row.get('Amenity_Lat_supermarket')):
        destinations_set.add((row['Amenity_Lon_supermarket'], row['Amenity_Lat_supermarket']))
    if pd.notna(row.get('Amenity_Lon_train_station')) and pd.notna(row.get('Amenity_Lat_train_station')):
        destinations_set.add((row['Amenity_Lon_train_station'], row['Amenity_Lat_train_station']))

destinations_set.add(CBD_COORDS)
destinations_list = list(destinations_set)

max_origins_per_batch = max(1, MAX_ROUTES // len(destinations_list))

# Process batches
for origin_chunk, idx_chunk in zip(chunk_list(origins_list, max_origins_per_batch),
                                   chunk_list(property_indices, max_origins_per_batch)):
    print(f"Processing batch of {len(origin_chunk)} properties...")
    
    res_matrix = safe_distance_matrix(origin_chunk, destinations_list)
    distances_matrix = res_matrix['distances']
    durations_matrix = res_matrix['durations']
    
    for i, prop_idx in enumerate(idx_chunk):
        row = amenities_pivot.loc[prop_idx]
        dist_supermarket = dist_train = dist_CBD = float('nan')
        time_supermarket = time_train = time_CBD = float('nan')
        
        for j, dest in enumerate(destinations_list):
            if (pd.notna(row.get('Amenity_Lon_supermarket')) and pd.notna(row.get('Amenity_Lat_supermarket')) and
                dest == (row['Amenity_Lon_supermarket'], row['Amenity_Lat_supermarket'])):
                dist_supermarket = distances_matrix[i][j]
                time_supermarket = durations_matrix[i][j]
            if (pd.notna(row.get('Amenity_Lon_train_station')) and pd.notna(row.get('Amenity_Lat_train_station')) and
                dest == (row['Amenity_Lon_train_station'], row['Amenity_Lat_train_station'])):
                dist_train = distances_matrix[i][j]
                time_train = durations_matrix[i][j]
            if dest == CBD_COORDS:
                dist_CBD = distances_matrix[i][j]
                time_CBD = durations_matrix[i][j]
        
        # If row exists in distance_df, overwrite it
        if prop_idx in distance_df.index:
            distance_df.loc[prop_idx, ['Distance_to_supermarket_m','TravelTime_to_supermarket_s',
                                       'Distance_to_train_station_m','TravelTime_to_train_station_s',
                                       'Distance_to_CBD_m','TravelTime_to_CBD_s']] = [
                dist_supermarket, time_supermarket, dist_train, time_train, dist_CBD, time_CBD
            ]
        else:
            distance_df.loc[prop_idx] = {
                'Property_Lat': row['Property_Lat'],
                'Property_Lon': row['Property_Lon'],
                'Address': row.get('Address', ''),
                'Distance_to_supermarket_m': dist_supermarket,
                'TravelTime_to_supermarket_s': time_supermarket,
                'Distance_to_train_station_m': dist_train,
                'TravelTime_to_train_station_s': time_train,
                'Distance_to_CBD_m': dist_CBD,
                'TravelTime_to_CBD_s': time_CBD
            }
    
    # Save progress after each batch
    distance_df.to_csv(OUTPUT_CSV, index=False)
    print(f"Saved progress for {len(distance_df)} properties.")
    
    time.sleep(BATCH_DELAY)

print(" All batches processed.")

Resuming from 8661 completed properties...
Processing batch of 10 properties...
Saved progress for 8671 properties.
Processing batch of 10 properties...
Saved progress for 8681 properties.
Processing batch of 10 properties...
Saved progress for 8691 properties.
Processing batch of 10 properties...
Saved progress for 8701 properties.
Processing batch of 10 properties...
Saved progress for 8711 properties.
Processing batch of 10 properties...
Saved progress for 8721 properties.
Processing batch of 10 properties...
Saved progress for 8731 properties.
Processing batch of 10 properties...
Saved progress for 8741 properties.
Processing batch of 10 properties...
Saved progress for 8751 properties.
Processing batch of 10 properties...
Saved progress for 8761 properties.
Processing batch of 10 properties...
Saved progress for 8771 properties.
Processing batch of 10 properties...
Saved progress for 8781 properties.
Processing batch of 10 properties...
Saved progress for 8791 properties.
Processi

/Users/tasneemzulaiqa/Documents/GitHub/project-2-group-real-estate-industry-project-7-2025/proj2_env/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/tasneemzulaiqa/Documents/GitHub/project-2-group-real-estate-industry-project-7-2025/proj2_env/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/tasneemzulaiqa/Documents/GitHub/project-2-group-real-estate-industry-project-7-2025/proj2_env/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/tasneemzulaiqa/Documents/GitHub/project

Saved progress for 8841 properties.
Processing batch of 10 properties...
Saved progress for 8851 properties.
Processing batch of 10 properties...
Saved progress for 8861 properties.
Processing batch of 10 properties...
Saved progress for 8871 properties.
Processing batch of 10 properties...
Saved progress for 8881 properties.
Processing batch of 10 properties...
Saved progress for 8891 properties.
Processing batch of 10 properties...
Saved progress for 8901 properties.
Processing batch of 10 properties...
Saved progress for 8911 properties.
Processing batch of 10 properties...
Saved progress for 8921 properties.
Processing batch of 10 properties...
Saved progress for 8931 properties.
Processing batch of 10 properties...
Saved progress for 8941 properties.
Processing batch of 10 properties...
Saved progress for 8951 properties.
Processing batch of 10 properties...
Saved progress for 8961 properties.
Processing batch of 10 properties...
Saved progress for 8971 properties.
Processing batc

/Users/tasneemzulaiqa/Documents/GitHub/project-2-group-real-estate-industry-project-7-2025/proj2_env/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/tasneemzulaiqa/Documents/GitHub/project-2-group-real-estate-industry-project-7-2025/proj2_env/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/tasneemzulaiqa/Documents/GitHub/project-2-group-real-estate-industry-project-7-2025/proj2_env/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/tasneemzulaiqa/Documents/GitHub/project

Saved progress for 9161 properties.
Processing batch of 10 properties...
Saved progress for 9171 properties.
Processing batch of 10 properties...
Saved progress for 9181 properties.
Processing batch of 10 properties...
Saved progress for 9191 properties.
Processing batch of 10 properties...
Saved progress for 9201 properties.
Processing batch of 10 properties...
Saved progress for 9211 properties.
Processing batch of 10 properties...
Saved progress for 9221 properties.
Processing batch of 10 properties...
Saved progress for 9231 properties.
Processing batch of 10 properties...
Saved progress for 9241 properties.
Processing batch of 10 properties...
Saved progress for 9251 properties.
Processing batch of 10 properties...
Saved progress for 9261 properties.
Processing batch of 10 properties...
Saved progress for 9271 properties.
Processing batch of 10 properties...
Saved progress for 9281 properties.
Processing batch of 10 properties...
Saved progress for 9291 properties.
Processing batc

...

Saved progress for 10411 properties.
Processing batch of 10 properties...


/Users/tasneemzulaiqa/Documents/GitHub/project-2-group-real-estate-industry-project-7-2025/proj2_env/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/tasneemzulaiqa/Documents/GitHub/project-2-group-real-estate-industry-project-7-2025/proj2_env/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


Saved progress for 10421 properties.
Processing batch of 10 properties...


/Users/tasneemzulaiqa/Documents/GitHub/project-2-group-real-estate-industry-project-7-2025/proj2_env/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/tasneemzulaiqa/Documents/GitHub/project-2-group-real-estate-industry-project-7-2025/proj2_env/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


Saved progress for 10431 properties.
Processing batch of 10 properties...
Saved progress for 10441 properties.
Processing batch of 10 properties...
Saved progress for 10451 properties.
Processing batch of 10 properties...
Saved progress for 10461 properties.
Processing batch of 10 properties...
Saved progress for 10471 properties.
Processing batch of 10 properties...
Saved progress for 10481 properties.
Processing batch of 10 properties...
Saved progress for 10491 properties.
Processing batch of 10 properties...
Saved progress for 10501 properties.
Processing batch of 10 properties...
Saved progress for 10511 properties.
Processing batch of 10 properties...
Saved progress for 10521 properties.
Processing batch of 10 properties...
Saved progress for 10531 properties.
Processing batch of 10 properties...
Saved progress for 10541 properties.
Processing batch of 10 properties...
Saved progress for 10551 properties.
Processing batch of 10 properties...
Saved progress for 10561 properties.
P

### Try Google maps api (free trial)

In [ ]:

# Config
GMAPS_KEY = ""
CBD_COORDS = (-37.8183, 144.9671)  
BATCH_DELAY = 1  
MAX_RETRIES = 3

INPUT_CSV = amenities_pivot
OUTPUT_CSV = "../../datasets/property/property_with_distances_copy.csv"


# Initialize client
gmaps = googlemaps.Client(key=GMAPS_KEY)


# utility func
def safe_distance_matrix(origins, destinations):
    for attempt in range(MAX_RETRIES):
        try:
            result = gmaps.distance_matrix(origins, destinations, mode="driving")
            return result
        except Exception as e:
            print(f"Google Maps attempt {attempt+1} failed: {e}. Retrying in 5s...")
            time.sleep(5)
    print("⚠️ Google Maps failed after retries. Returning NaN.")
    return {
        "rows": [{"elements": [{"distance": {"value": float('nan')},
                                "duration": {"value": float('nan')}} for _ in destinations]} 
                 for _ in origins]
    }


# load/create output
if os.path.exists(OUTPUT_CSV):
    distance_df = pd.read_csv(OUTPUT_CSV)
    completed_indices = set(distance_df[~distance_df['Distance_to_CBD_m'].isna()].index)
    print(f"Resuming from {len(completed_indices)} completed properties...")
else:
    distance_df = pd.DataFrame()
    completed_indices = set()

# Process each property
for idx, row in amenities_pivot.iterrows():
    if idx in completed_indices:
        continue

    origins = [f"{row['Property_Lat']},{row['Property_Lon']}"]
    destinations = [f"{CBD_COORDS[0]},{CBD_COORDS[1]}"]

    if pd.notna(row.get('Amenity_Lat_supermarket')) and pd.notna(row.get('Amenity_Lon_supermarket')):
        destinations.append(f"{row['Amenity_Lat_supermarket']},{row['Amenity_Lon_supermarket']}")
    if pd.notna(row.get('Amenity_Lat_train_station')) and pd.notna(row.get('Amenity_Lon_train_station')):
        destinations.append(f"{row['Amenity_Lat_train_station']},{row['Amenity_Lon_train_station']}")

    res_matrix = safe_distance_matrix(origins, destinations)
    elements = res_matrix["rows"][0]["elements"]

    # Assign distances/durations
    dist_CBD = elements[0]["distance"]["value"]
    time_CBD = elements[0]["duration"]["value"]
    dist_supermarket = time_supermarket = dist_train = time_train = float('nan')

    for i, dest in enumerate(destinations[1:]):
        if dest == f"{row.get('Amenity_Lat_supermarket')},{row.get('Amenity_Lon_supermarket')}":
            dist_supermarket = elements[i+1]["distance"]["value"]
            time_supermarket = elements[i+1]["duration"]["value"]
        elif dest == f"{row.get('Amenity_Lat_train_station')},{row.get('Amenity_Lon_train_station')}":
            dist_train = elements[i+1]["distance"]["value"]
            time_train = elements[i+1]["duration"]["value"]

    # Save row
    distance_df.loc[idx] = {
        'Property_Lat': row['Property_Lat'],
        'Property_Lon': row['Property_Lon'],
        'Address': row.get('Address', ''),
        'Distance_to_supermarket_m': dist_supermarket,
        'TravelTime_to_supermarket_s': time_supermarket,
        'Distance_to_train_station_m': dist_train,
        'TravelTime_to_train_station_s': time_train,
        'Distance_to_CBD_m': dist_CBD,
        'TravelTime_to_CBD_s': time_CBD
    }

    # Save progress after each property
    distance_df.to_csv(OUTPUT_CSV, index=False)
    print(f"Saved property {idx}")
    time.sleep(BATCH_DELAY)


Resuming from 2594 completed properties...
Saved property 2594
Saved property 2595
Saved property 2596
Saved property 2597
Saved property 2598
Saved property 2599
Saved property 2600
Saved property 2601
Saved property 2602
Saved property 2603
Saved property 2604
Saved property 2605
Saved property 2606
Saved property 2607
Saved property 2608
Saved property 2609
Saved property 2610
Saved property 2611
Saved property 2612
Saved property 2613
Saved property 2614
Saved property 2615
Saved property 2616
Saved property 2617
Saved property 2618
Saved property 2619
Saved property 2620
Saved property 2621
Saved property 2622
Saved property 2623
Saved property 2624
Saved property 2625
Saved property 2626
Saved property 2627
Saved property 2628
Saved property 2629
Saved property 2630
Saved property 2631
Saved property 2632
Saved property 2633
Saved property 2634
Saved property 2635
Saved property 2636
Saved property 2637
Saved property 2638
Saved property 2639
Saved property 2640
Saved property 26

KeyboardInterrupt: 